# A Selenium-Based Web Scraper to scrape annual report on cninfo.com

collect data into format:
- code
- name
- title
- link

**primary key**: (code, name)

In [16]:
import os
import pandas as pd
import numpy as np
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import quote
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import json
from selenium.webdriver.common.action_chains import ActionChains
import time
from urllib.parse import quote
import json

## 1. Initialize Webdriver

In [17]:
driver_path = r"C:\Program Files\Google\chromedriver-win64\chromedriver.exe" # your driver path
service = Service(executable_path=driver_path)
options = webdriver.ChromeOptions()
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
)

In [18]:
url = 'http://www.cninfo.com.cn/new/commonUrl/pageOfSearch?url=disclosure/list/search&lastPage=index' # url of cninfo.com homepage
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
wait = WebDriverWait(driver, 10)
driver.maximize_window()

## 2. Customize Searching Keywords

In [5]:
# 选择年报
nb_element = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[@title="年报" and contains(@class, "cate-checkbox-txt")]')))
nb_element.click()

In [ ]:
# 点击选择日期输入框
time.sleep(2)
date_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".el-range-input:nth-child(2)")))
date_input.click()
# 选择快捷日期——近三年
quick_date = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".el-picker-panel__shortcut:nth-child(8)")))
quick_date.click()

In [8]:
# 选择行业
# 点击搜索按钮
search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".el-col:nth-child(4) .el-button")))
search_button.click()

In [9]:
# 等待复选框出现
checkboxes = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".el-checkbox-group .el-checkbox")))

# 选择“制造业”
for checkbox in checkboxes:
    if checkbox.text.strip() == "制造业":
        checkbox.click()
        break

# 等待一段时间以确保操作生效
driver.implicitly_wait(2)

# 退出复选框（点击页面其他部分退出）
body_element = driver.find_element(By.TAG_NAME, "body")
body_element.click()

In [10]:
# 点击搜索
query_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".query-btn")))
query_button.click()

## 3. Scrape Web Content
+ “年度报告摘要”排除
+ 附加一列链接

### 3.1 爬取单页

In [43]:
# # 等待表格元素加载完成
# table_locator = (By.CSS_SELECTOR, '#main > div.jc-layout.tab-ct > div:nth-child(1) > div.el-col.el-col-24.el-col-md-17 > div.el-table-box.el-table-box-history')
# wait = WebDriverWait(driver, 10)
# table = wait.until(EC.presence_of_element_located(table_locator))

In [44]:
# # 提取表格列名
# headers = [header.text for header in table.find_elements(By.XPATH, ".//th")][:4]

In [45]:
# data = []

In [46]:
# rows = table.find_elements(By.XPATH, ".//tbody/tr")
# for row in rows:
#     # 获取行数据
#     row_data = [cell.text for cell in row.find_elements(By.XPATH, ".//td")]

#     # 判断公告标题中是否包含“摘要”，如果包含则跳过当前行
#     if '摘要' in row_data[headers.index('公告标题')]:
#         continue

#     # 提取第三列“公告标题”列的链接——即年报文本链接
#     link = row.find_elements(By.XPATH, ".//td")[headers.index('公告标题')].find_element(By.XPATH, ".//a").get_attribute('href')

#     # 添加链接到行数据中
#     row_data.append(link)
#     data.append(row_data)
#     #print(row_data)
# # data

In [47]:
# 创建 DataFrame
# df = pd.DataFrame(data, columns=headers + ['link'])
# df.drop(columns=['公告时间'], inplace=True)
# df

,代码,简称,公告标题,link
0,603688,石英股份,江苏太平洋石英股份有限公司2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
1,600563,法拉电子,法拉电子2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
2,300999,金龙鱼,2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
3,688323,瑞华泰,瑞华泰：2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
4,600589,*ST榕泰,关于收到上海证券交易所对公司2023年年度报告的信息披露问询函的公告,http://www.cninfo.com.cn/new/disclosure/detail...
5,600055,万东医疗,万东医疗2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
6,600750,江中药业,江中药业2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
7,600075,新疆天业,新疆天业股份有限公司2023年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
8,002519,银河电子,2023年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
9,600075,新疆天业,申万宏源证券承销保荐有限责任公司关于新疆天业股份有限公司2023年度持续督导年度报告书,http://www.cninfo.com.cn/new/disclosure/detail...


### 3.2 Turn to Next Page

In [42]:
# # 点击下一页按钮
# next_page_button = driver.find_element(By.CSS_SELECTOR, "#main > div.jc-layout.tab-ct > div:nth-child(1) > div.el-col.el-col-24.el-col-md-17 > div.new-pagination > div > button.btn-next")
# next_page_button.click()

## 4. Wrap Up

In [11]:
# 等待表格元素加载完成
table_locator = (By.CSS_SELECTOR, '#main > div.jc-layout.tab-ct > div:nth-child(1) > div.el-col.el-col-24.el-col-md-17 > div.el-table-box.el-table-box-history')
wait = WebDriverWait(driver, 10)
table = wait.until(EC.presence_of_element_located(table_locator))
# 提取表格列名
headers = [header.text for header in table.find_elements(By.XPATH, ".//th")][:4]

In [12]:
# Loop：爬取《——》翻页
all_data=[]
for i in range(10): # e.g. sample 爬取前10页
    # 爬取内容
    # 等待表格元素加载完成
    table_locator = (By.CSS_SELECTOR, '#main > div.jc-layout.tab-ct > div:nth-child(1) > div.el-col.el-col-24.el-col-md-17 > div.el-table-box.el-table-box-history')
    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located(table_locator))
    rows = table.find_elements(By.XPATH, ".//tbody/tr")
    data=[]
    for row in rows:
        # 获取行数据
        row_data = [cell.text for cell in row.find_elements(By.XPATH, ".//td")]

        # 判断公告标题中是否包含“摘要”，如果包含则跳过当前行
        if '摘要' in row_data[headers.index('公告标题')]:
            continue

        # 提取第三列“公告标题”列的链接——即年报文本链接
        link = row.find_elements(By.XPATH, ".//td")[headers.index('公告标题')].find_element(By.XPATH, ".//a").get_attribute('href')

        # 添加链接到行数据中
        row_data.append(link)
        data.append(row_data)
    # 如果表格数据不为空，则将其追加到all_data中
    if data:
        all_data.extend(data)

    # 点击下一页按钮
    next_page_button = driver.find_element(By.CSS_SELECTOR, "#main > div.jc-layout.tab-ct > div:nth-child(1) > div.el-col.el-col-24.el-col-md-17 > div.new-pagination > div > button.btn-next")
    next_page_button.click()

    print(f'----已完成{i+1}页----')

    # 等待新页面加载完成
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//table[@class='table']")))


----已完成1页----
----已完成2页----
----已完成3页----
----已完成4页----
----已完成5页----
----已完成6页----
----已完成7页----
----已完成8页----
----已完成9页----
----已完成10页----


In [13]:
# 创建 DataFrame
df = pd.DataFrame(all_data, columns=headers + ['link'])
df.drop(columns=['公告时间'], inplace=True)
df.rename(columns={'代码':'code', '简称':'name', '公告标题':'title'}, inplace=True)
df

,code,name,title,link
0,603019,中科曙光,中科曙光2024年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
1,872808,曙光数创,2024年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
2,300353,东土科技,2024年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
3,000063,中兴通讯,2024年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
4,688041,海光信息,海光信息技术股份有限公司2024年年度报告,http://www.cninfo.com.cn/new/disclosure/detail...
...,...,...,...,...
223,603008,喜临门,喜临门家具股份有限公司2023年年度报告（更正后）,http://www.cninfo.com.cn/new/disclosure/detail...
224,300398,飞凯材料,2023年年度报告（更新后）,http://www.cninfo.com.cn/new/disclosure/detail...
225,300398,飞凯材料,2021年年度报告（更正后）,http://www.cninfo.com.cn/new/disclosure/detail...
226,300398,飞凯材料,2022年年度报告（更正后）,http://www.cninfo.com.cn/new/disclosure/detail...


In [15]:
# export to excel
df.to_csv(r'./Data/annual_report_10pages.csv',index=False)